## Validação cruzada do modelo 1


In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
livro = pd.read_csv('data/livro.csv')
livro

,intern_clock,1,2,3,4,x,y
0,31032,0.0,0.0,0.0,0.0,39.805067,-17.093603
1,31130,0.0,0.0,0.0,0.0,31.603003,-15.421540
2,31227,0.0,0.0,0.0,0.0,33.888520,-19.008985
3,31326,0.0,0.0,0.0,0.0,35.133377,-20.662502
4,31424,0.0,0.0,0.0,0.0,26.759043,-16.759596
...,...,...,...,...,...,...,...
699,99546,0.0,0.0,0.0,0.0,19.244610,26.384993
700,99643,0.0,0.0,0.0,0.0,19.725827,27.399102
701,99741,0.0,0.0,0.0,0.0,18.731329,28.161931
702,99840,0.0,0.0,0.0,0.0,18.748916,27.735171


In [2]:
# pra fazer os negócios funcionar
import nest_asyncio
nest_asyncio.apply()

# pra sumir com os warnings
import warnings
warnings.filterwarnings('ignore')  # não deu certo :(

In [3]:
from math import floor
from math import ceil

# tem que dar um shufle
livro = livro.sample(frac = 1)
size = livro.shape[0]

X_train = livro[['1', '2', '3', '4']][0:floor(size*70/100)].reset_index().drop('index', axis=1)
X_test = livro[['1', '2', '3', '4']][0:ceil(size*30/100)].reset_index().drop('index', axis=1)

x_train = livro['x'][0:floor(size*70/100)].reset_index().drop('index', axis=1).to_numpy().reshape(floor(size*70/100),)
x_test = livro['x'][0:floor(size*30/100)].reset_index().drop('index', axis=1).to_numpy().reshape(floor(size*30/100),)

y_train = livro['y'][0:floor(size*70/100)].reset_index().drop('index', axis=1).to_numpy().reshape(floor(size*70/100),)
y_test = livro['y'][0:floor(size*30/100)].reset_index().drop('index', axis=1).to_numpy().reshape(floor(size*30/100),)

Treinando o modelo para x

In [9]:
# From: Bayesian Models for Astrophysical Data, Cambridge Univ. Press
# (c) 2017,  Joseph M. Hilbe, Rafael S. de Souza and Emille E. O. Ishida 
# 
# you are kindly asked to include the complete citation if you used this 
# material in a publication

# Code 4.7 - Multivariate normal linear model in Python using Stan
# 1 response (y) and 2 explanatory variables (x1, x2)

# ^^ a implementação foi inspirada no autor acima

import numpy as np
import statsmodels.api as sm
import stan
from scipy.stats import uniform, norm
import json
# Data
np.random.seed(1056)                 # set seed to replicate example
nobs= X_train.shape[0]                         # number of obs in model 
x1 = X_train['1'].to_numpy()          # random uniform variable
x2 = X_train['2'].to_numpy()          # second explanatory
x3 = X_train['3'].to_numpy()
x4 = X_train['4'].to_numpy()

X = np.column_stack((x1,x2, x3, x4))        # create response matrix
# X = np.column_stack((x1, x2))
X = sm.add_constant(X)              # add intercept

y = x_train # agora faz regressão na abcissa do centro de massa calculado do corpo de prova

# Fit
data = {}                                # build data dictionary
data['nobs'] = nobs                      # sample size
data['x'] = X                            # explanatory variable         
data['y'] = y                            # response variable
data['k'] = data['x'].shape[1]       # number of explanatory variables

# data = json.dumps(list(data))

# data = data.to_json()


# STAN code
stan_code = """
data {
    int<lower=1> k;  
    int<lower=0> nobs;                                 
    matrix[nobs, k] x;                     
    vector[nobs] y;                     
}
parameters {
    matrix[k,1] beta;                                             
    real<lower=0> sigma;               
}
transformed parameters{
    matrix[nobs,1] mu;
    vector[nobs] mu2;

    mu = x * beta;
    mu2 = to_vector(mu);                 // normal distribution 
                                         // does not take matrices as input
}
model {
    for (i in 1:k){                      // Diffuse normal priors for predictors
        beta[i] ~ normal(0.0, 100);
    }
    sigma ~ gamma(32.168426055, 49.11855);           // A variância dos sensores nao é baixa mas eles são relativamente confiáveis
    
    y ~ normal(mu2, sigma);              // Likelihood function
}
"""

#  sigma ~ gamma(76.083434761, 75.539775);           




# Compile model
model = stan.build(stan_code, data)

# perform fit
fit1 = model.sample(num_chains=4, num_samples=1000)


# Output
nlines = 9                                   # number of lines in screen output

output = str(fit1).split('\n')
for item in output[:nlines]:
    print(item)   

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_6jv2fi4x/model_hpnrszw7.stan', line 24, column 32: Argument
    49.11855 suggests there may be parameters that are not unit scale;
    consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_6jv2fi4x/model_hpnrszw7.stan', line 24, column 18: Argument
    32.168426055 suggests there may be parameters that are not unit scale;
    consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_6jv2fi4x/model_hpnrszw7.stan', line 22, column 30: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:  26% (2102/8000)
Sampling:  51% (4101/8000)
Sampling:  75% (6001/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000

<stan.Fit>
Parameters:
    beta: (5, 1)
    sigma: ()
    mu: (492, 1)
    mu2: (492,)
Draws: 4000


In [10]:
import arviz as az

summ = az.summary(fit1)
summ = summ[:][0:6]
summ

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"beta[0, 0]",-24.557,9.524,-42.546,-6.730,0.140,0.101,4656.0,3377.0,1.0
"beta[1, 0]",1.751,44.652,-80.899,86.350,0.633,0.726,4964.0,2930.0,1.0
"beta[2, 0]",149.467,34.410,81.813,209.960,0.475,0.342,5247.0,3435.0,1.0
"beta[3, 0]",-0.292,100.243,-189.037,190.803,1.372,1.526,5341.0,3196.0,1.0
"beta[4, 0]",-1.301,104.042,-205.678,182.538,1.441,1.901,5212.0,2456.0,1.0
sigma,204.485,1.155,202.358,206.653,0.015,0.010,6168.0,3133.0,1.0


In [11]:
coefsx = summ['mean'][0:6]
coefsx

beta[0, 0]    -24.557
beta[1, 0]      1.751
beta[2, 0]    149.467
beta[3, 0]     -0.292
beta[4, 0]     -1.301
sigma         204.485
Name: mean, dtype: float64

In [12]:
Treinando o modelo para y

SyntaxError: invalid syntax (3992363052.py, line 1)

In [13]:
# From: Bayesian Models for Astrophysical Data, Cambridge Univ. Press
# (c) 2017,  Joseph M. Hilbe, Rafael S. de Souza and Emille E. O. Ishida 
# 
# you are kindly asked to include the complete citation if you used this 
# material in a publication

# Code 4.7 - Multivariate normal linear model in Python using Stan
# 1 response (y) and 2 explanatory variables (x1, x2)

# ^^ a implementação foi inspirada no autor acima

import numpy as np
import statsmodels.api as sm
import stan
from scipy.stats import uniform, norm
import json
# Data
np.random.seed(1056)                 # set seed to replicate example
nobs= X_train.shape[0]                         # number of obs in model 
x1 = X_train['1'].to_numpy()          # random uniform variable
x2 = X_train['2'].to_numpy()          # second explanatory
x3 = X_train['3'].to_numpy()
x4 = X_train['4'].to_numpy()

X = np.column_stack((x1,x2, x3, x4))        # create response matrix
# X = np.column_stack((x1, x2))
X = sm.add_constant(X)              # add intercept

y = y_train # agora faz regressão na abcissa do centro de massa calculado do corpo de prova

# Fit
data = {}                                # build data dictionary
data['nobs'] = nobs                      # sample size
data['x'] = X                            # explanatory variable         
data['y'] = y                            # response variable
data['k'] = data['x'].shape[1]       # number of explanatory variables

# data = json.dumps(list(data))

# data = data.to_json()


# STAN code
stan_code = """
data {
    int<lower=1> k;  
    int<lower=0> nobs;                                 
    matrix[nobs, k] x;                     
    vector[nobs] y;                     
}
parameters {
    matrix[k,1] beta;                                             
    real<lower=0> sigma;               
}
transformed parameters{
    matrix[nobs,1] mu;
    vector[nobs] mu2;

    mu = x * beta;
    mu2 = to_vector(mu);                 // normal distribution 
                                         // does not take matrices as input
}
model {
    for (i in 1:k){                      // Diffuse normal priors for predictors
        beta[i] ~ normal(0.0, 100);
    }
    sigma ~ gamma(76.083434761, 75.539775);           // A variância dos sensores nao é baixa mas eles são relativamente confiáveis
    
    y ~ normal(mu2, sigma);              // Likelihood function
}
"""

#  sigma ~ gamma(76.083434761, 75.539775);           




# Compile model
model = stan.build(stan_code, data)

# perform fit
fit2 = model.sample(num_chains=4, num_samples=1000)


# Output
nlines = 9                                   # number of lines in screen output

output = str(fit1).split('\n')
for item in output[:nlines]:
    print(item)   

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_00zm6ej4/model_llcrskhi.stan', line 24, column 32: Argument
    75.539775 suggests there may be parameters that are not unit scale;
    consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_00zm6ej4/model_llcrskhi.stan', line 24, column 18: Argument
    76.083434761 suggests there may be parameters that are not unit scale;
    consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_00zm6ej4/model_llcrskhi.stan', line 22, column 30: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:  25% (2003/8000)
Sampling:  50% (4002/8000)
Sampling:  75% (6001/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages receive

<stan.Fit>
Parameters:
    beta: (5, 1)
    sigma: ()
    mu: (492, 1)
    mu2: (492,)
Draws: 4000


In [14]:
import arviz as a

summ = az.summary(fit2)
summ = summ[:][0:6]
summ

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"beta[0, 0]",15.416,1.487,12.777,18.361,0.025,0.017,3695.0,3100.0,1.0
"beta[1, 0]",0.104,7.749,-13.959,14.987,0.110,0.119,4989.0,3128.0,1.0
"beta[2, 0]",-0.053,5.642,-10.371,10.287,0.084,0.088,4528.0,3315.0,1.0
"beta[3, 0]",-0.861,99.873,-193.762,182.026,1.431,1.596,4908.0,3056.0,1.0
"beta[4, 0]",-2.649,99.833,-192.970,178.894,1.376,1.755,5279.0,3087.0,1.0
sigma,31.541,0.358,30.901,32.260,0.005,0.003,5612.0,3196.0,1.0


In [15]:
coefsy = summ['mean'][0:6]
coefsy[0]

15.416

definindo funções para validação cruzada:

In [19]:
# avaliando erro quadrático médio
import math

def EQM(c0, c1, c2, c3, c4, X_test, y_test):
    
    eqm = 0
    size = len(y_test)
    for i in range(size):
        
        eqm += (((c0 + c1*X_test['1'][i] + c2*X_test['2'][i] + c3*X_test['3'][i] + c4*X_test['4'][i]) - y_test[i])**2)/size
        
    return eqm 

# avaliando erro absoluto médio

def EAM(c0, c1, c2, c3, c4, X_test, y_test):
    
    eam = 0
    size = len(y_test)
    for i in range(size):
        
        eam += abs(c0 + c1*X_test['1'][i] + c2*X_test['2'][i] + c3*X_test['3'][i] + c4*X_test['4'][i] - y_test[i])/size
        
    return eam 

In [17]:
EQM(coefsx[0], coefsx[1], coefsx[2], coefsx[3], coefsx[4], X_test, x_test)

1889850.724903651

In [20]:
EAM(coefsx[0], coefsx[1], coefsx[2], coefsx[3], coefsx[4], X_test, x_test)

187.53377456810912

In [18]:
EAM(coefsy[0], coefsy[1], coefsy[2], coefsy[3], coefsy[4], X_test, y_test)

10060.953041774517

In [21]:
EAM(coefsy[0], coefsy[1], coefsy[2], coefsy[3], coefsy[4], X_test, y_test)

31.638794201218968

In [23]:
# avaliando quantos elementos estão dentro da regiao de confiança mutua

def conf(coefsx, coefsy, lamb, X_test, x_test, y_test):
     
    acertos = 0
    
    
    for i in range(len(y_test)): 
    
        x_prev = coefsx[0] + coefsx[1]*X_test['1'][i] + coefsx[2]*X_test['2'][i] + coefsx[3]*X_test['3'][i] + coefsx[4]*X_test['4'][i]
        y_prev = coefsy[0] + coefsy[1]*X_test['1'][i] + coefsy[2]*X_test['2'][i] + coefsy[3]*X_test['3'][i] + coefsy[4]*X_test['4'][i]
    
        if (x_test[i] > x_prev - lamb*coefsx[5] and x_test[i] < x_prev + lamb*coefsx[5] and
            y_test[i] > y_prev - lamb*coefsy[5] and y_test[i] < y_prev + lamb*coefsy[5]):

            acertos += 1
            
    return acertos/len(y_test)

In [24]:
lamb = 2

lambs = [0.1, 0.2, 0.5, 1, 1.5, 2]
acertos = [0]*len(lambs)

for i in range(len(lambs)):
    acertos[i] = conf(coefsx, coefsy, lambs[i], X_test, x_test, y_test)

acertos

[0.004739336492890996,
 0.061611374407582936,
 0.3933649289099526,
 0.8293838862559242,
 0.8957345971563981,
 0.9241706161137441]